In [4]:
import pandas as pd
import urllib
import time
import json

    

In [5]:
studentBehavior = pd.read_json("studentBehaviorInfoOver40Class_1213.json", orient="index")

In [6]:
print(studentBehavior[0])
# print studentBehavior[0]["memberId"]
# print (studentBehavior[0]["chosenVideo"])
print (studentBehavior[0]["listenScore"])


## Desired json structure
# [
#     {
#         "memberId": 12345,
#         "videoSequence":
#             [
#                 3980,
#                 7680,
#                 1235
#             ],
#         "skippingRatioSequence":
#             [
#                 0,
#                 0,
#                 0.3
#             ],
#         "scoreSequence":
#             [
#                 80,
#                 70,
#                 75
#             ],
#         "dictionarySequence":
#             [
#                 0,
#                 0,
#                 0
#             ],
#         "skippingIndex": 10,
#         "clusterSequence":
#             [
#                 "C1",
#                 "C1",
#                 "C2"
#             ]
        
#     }
# ]



chosenVideo        [3913, 5186, 4974, 4802, 3711, 5881, 7126, 5797]
listenScore       [{'postId': 3913, 'score': 93}, {'postId': 391...
memberId                                                      50679
vocabularyList    [{'postId': '5797', 'word': 'devil'}, {'postId...
Name: 0, dtype: object
[{'postId': 3913, 'score': 93}, {'postId': 3913, 'score': 91}, {'postId': 3913, 'score': 92}, {'postId': 3913, 'score': 92}, {'postId': 3913, 'score': 92}, {'postId': 5186, 'score': 93}, {'postId': 5186, 'score': 94}, {'postId': 5186, 'score': 93}, {'postId': 4974, 'score': 90}, {'postId': 4974, 'score': 80}, {'postId': 4974, 'score': 77}, {'postId': 4974, 'score': 96}, {'postId': 4974, 'score': 86}, {'postId': 4974, 'score': 97}, {'postId': 4974, 'score': 90}, {'postId': 4974, 'score': 95}, {'postId': 4974, 'score': 95}, {'postId': 4974, 'score': 98}, {'postId': 4974, 'score': 91}, {'postId': 4974, 'score': 86}, {'postId': 4974, 'score': 95}, {'postId': 4974, 'score': 86}, {'postId': 4802, 'scor

In [24]:
# Generate Users Profile File

# This method return a mapping of video(key) 
# and and array of scores(value)
def parse_video_scores(scores_object):
    result = {}
    for score in scores_object:
        if score['postId'] not in result:
            result[score['postId']] = []
        result[score['postId']].append(score['score'])
    return result
            
def calc_skipping_ratio(scores):
    countSkipped = 0
    for score in scores:
        if score < 0:
            countSkipped += 1
    return round(countSkipped/len(scores),2)

def calc_avg_score(scores):
    countSkipped = 0
    sumScores = 0
    for score in scores:
        if score < 0:
            countSkipped += 1
        else:
            sumScores += score
                
    if len(scores)-countSkipped == 0:
        return 0
    else:
        return round(sumScores/(len(scores)-countSkipped),2)
    
lenStudentBehavior = len(pd.read_json("studentBehaviorInfoOver40Class_1213.json"))
usersProfilesJsonWithIndexes = {}
for index in range(lenStudentBehavior):
    memberId = studentBehavior[index]["memberId"]
    if memberId not in usersProfilesJsonWithIndexes:
        studentProfile                          = {}
        studentProfile["memberId"]              = memberId
        studentProfile["videoSequence"]         = []
        studentProfile["skippingRatioSequence"] = []
        studentProfile["avgScoreSequence"]         = []
        studentProfile["dictionarySequence"]    = []
        studentProfile["skippingIndex"]         = -1
        studentProfile["clusterSequence"]       = []
        usersProfilesJsonWithIndexes[memberId] = studentProfile
    
    # Addid values for every key
    # Video Sequence
    usersProfilesJsonWithIndexes[memberId]["videoSequence"].extend(studentBehavior[index]["chosenVideo"])
    
    
    videos_scores = parse_video_scores(studentBehavior[index]["listenScore"])
    
    # SkippingRationSequence
    for video in studentBehavior[index]["chosenVideo"]:
        scores = videos_scores[video]
        usersProfilesJsonWithIndexes[memberId]["skippingRatioSequence"].append(calc_skipping_ratio(scores))
        # Average Score Sequence
        avg = calc_avg_score(scores)
        usersProfilesJsonWithIndexes[memberId]["avgScoreSequence"].append(avg)
        
    # DictionarySequence
    
        
    # skippingIndex
    skip_index = [ n for n,i in enumerate(usersProfilesJsonWithIndexes[memberId]["skippingRatioSequence"]) if i>0.0 ][0]
    usersProfilesJsonWithIndexes[memberId]["skippingIndex"] = skip_index
        
    # ClusterSequece
    

fileStudentProfile = open("studentProfile.json", "w")
fileStudentProfile.write("[")
lenObjects = len(usersProfilesJsonWithIndexes)
it = 0
for key, value in usersProfilesJsonWithIndexes.items():
    json.dump(value,fileStudentProfile)
    if it != (lenObjects-1):
        fileStudentProfile.write(", ")
    it = it + 1
    


fileStudentProfile.write("]")
fileStudentProfile.close()

# Notes

1. We have three different skipRatio behavior
    User that goes from 0 to 1
    Users that goes from 0 to [a range gratter than 0] and stop
    Users that goes from 0 to [a range gratter than 0] and then skippRation 1

2. The skipIndex shows the index where the skipRation is gratter than 0
    
